# Week5 Advanced 심화과제

- 논문요약

    1) 필수 지시사항은 논문을 요약하는 것입니다.

    2) 이를 조금 확장시켜, rag에 관한 여러가지 논문을 읽어와 rag에 관한 질문은 검색된 문서 기반으로 대답하고 일상 대화는 context에 상관없이 대답하는 챗봇을 만들 예정입니다.

    3) 주어진 하나의 논문을 요약하는 것이 아닌 질의 응답을 통해 rag에 관한 개념 설명을 로그로 남기는 것을 목표로 합니다.

- 요약 논문: arXiv에 게제된 rag에 관한 논문 9편



## 1 [My Code] Preparation

In [2]:
import os
import openai
from dotenv import load_dotenv

load_dotenv(dotenv_path="config.env")
openai.api_key = os.getenv("OPENAI_API_KEY")

### 1.1 [My Code] Data Load

- pdf를 불러올 땐 unstructuredfileloader를 사용합니다

In [3]:
from pathlib import Path

from langchain.document_loaders import UnstructuredFileLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

# pdf를 불러올 땐 unstructuredFileLoader를 사용합니다
path = Path('./data/essay')
documents = [UnstructuredFileLoader(file_path).load() for file_path in path.glob('*.pdf')]


/tmp/ipykernel_1072/3843052403.py:8: LangChainDeprecationWarning: The class `UnstructuredFileLoader` was deprecated in LangChain 0.2.8 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-unstructured package and should be used instead. To use it run `pip install -U :class:`~langchain-unstructured` and import as `from :class:`~langchain_unstructured import UnstructuredLoader``.
  documents = [UnstructuredFileLoader(file_path).load() for file_path in path.glob('*.pdf')]


### 1.2 [My Code] TextSplitter

In [4]:
# 1000개 길이를 기준으로 문서를 자르고 200개 길이만큼 겹치도록 문서를 구성합니다.
splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splitted_docs = []
for document in documents:
    splitted_docs.extend(splitter.split_documents(document))

### 1.3 [My Code] 문장 생성 모델 선언

In [ ]:
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain_openai import ChatOpenAI

# 문장 생성기
# 타 모델과의 비교도 가능하지만, 챗봇 구현시엔 해당 모델로 고정해두고 사용합니다. 
llm = ChatOpenAI(
    model='gpt-4o-mini',
    temperature=0.7,
    max_tokens=1024,
    )

### 1.4 [My Code] 임베딩 모델 선언

- 여기선 CacheBackedEmbeddings을 사용하여, 인풋이 같을 때엔 다시 임베딩을 하지 않고 캐시된 정보를 사용하도록 수정합니다.,

In [6]:
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings
from langchain.embeddings import CacheBackedEmbeddings
from langchain.storage import LocalFileStore


# OpenAI Embeddings 초기화
# https://platform.openai.com/docs/guides/embeddings
# 임베딩 전용
embeddings = OpenAIEmbeddings(
    api_key=openai.api_key,
    model="text-embedding-3-small"
)

# 이렇게 하면 동일한 파일에 대해서, 매번 임베딩하지 않습니다.
cache_dir = LocalFileStore(".cache/files")
cached_embeddings = CacheBackedEmbeddings.from_bytes_store(
    embeddings, cache_dir
)

# Chroma 벡터 저장소 생성 및 로컬 저장 경로 지정
vectorstore = Chroma.from_documents(
    documents=splitted_docs,
    embedding=cached_embeddings,
    collection_name="my_db"
)


# 유사 문서는 20개를 찾도록 합니다.
# search_type에 similarity와 mmr이 보통 많이 사용되는데 여기선 similarity를 기준으로 하겠습니다. 
chroma_retriever = vectorstore.as_retriever(search_type='similarity', search_kwargs={'k': 20})

### 1.5 [My Code] 프롬프트

- LLM 교수와 학생의 질의응답 상황을 가정합니다.

- rag와 관련 없는 질문은 context와 상관없이 대답하며 rag와 관련된 질문은 context 내에서 답변하도록 prompt를 구성합니다.

In [7]:
from langchain.prompts import ChatPromptTemplate
# 반드시 프롬프트는 chatprompttemplate 객체에 넣어야 합니다.
prompt_template = """
당신은 대학 LLM 교수입니다. 아래 지침에 따라 학생의 질문에 답해주세요.

1. 문맥에서 답을 명확히 찾아내야 하며, **rag(Relevant Answer Generation)**에 관한 질문은 반드시 제공된 문맥에만 근거해서 답변해야 합니다.  
   - rag에 관한 질문에서 문맥만으로 답을 찾을 수 없으면, "아직 저도 부족해서 공부가 더 필요할 거 같아요. 같이 공부해봐요."라고 대답하세요.  
2. rag와 관련 없는 일상적인 대화에는 자유롭게 답변하되, 친절하고 대화체로 작성하세요.  
3. 모든 답변은 5문장을 넘지 않아야 하며, rag 관련 답변에는 반드시 "잘 이해되었나요?"라는 질문을 포함하세요.  
4. 학생이 이해하기 쉽게 설명하며, 항상 친절한 교수님의 톤을 유지하세요.

문맥:  
{context}

학생의 질문:  
{question}

답변:
"""
chat_prompt_template = ChatPromptTemplate.from_template(prompt_template)

## 2 [My Code] 챗봇

- 문맥은 5개만 기억하도록 설정합니다.

- streamlit 등에선 따로 session이 존재하지만 파이썬 자체로 구현하기 위해 deque 데이터타입을 사용합니다.

- 원래는, 기존의 모든 대화에서 query와 유사한 답변을 추출하여 세션으로 넘기는 로직 또한 포함되어야 하지만, 토이프로젝트에선 이를 생략하고 최근접 대화 5개만 세션으로 넘기도록 하겠습니다.

In [8]:
from collections import deque

class ChatBot:
    def __init__(self, retriever, llm, prompt_template, history_limit=5):
        self.retriever = retriever
        self.llm = llm
        self.prompt_template = prompt_template
        self.history = deque(maxlen=history_limit)

    # docs: Document 객체의 list
    @staticmethod
    def _format_docs(docs):
        return "\n\n".join(f"Essay Source: {doc.metadata['source'].split('/')[-1]}, Content: {doc.page_content}" for doc in docs)
    
    def create_prompt(self, user_message):
        # 처음엔 context가 없어서 None으로 들어갑니다.
        # 따라서 초반 서사가 쌓이기 전까진 retrieved_docs만 들어갑니다.
        context = "\n\n".join([f"Student: {msg['Student']}\nLLM Professor: {msg['LLM Professor']}" for msg in self.history])
        retrieved_docs = self.retriever.invoke(user_message)

        # 반환된 document를 전처리합니다.
        retrieved_context = ChatBot._format_docs(retrieved_docs)
        
        return self.prompt_template.invoke({
            "context": context + "\n\n" + retrieved_context,
            "question": user_message
        })

    def chat(self, user_message):
        user_message = user_message.lower()
        user_prompt = self.create_prompt(user_message)
        response = self.llm.invoke(user_prompt)
        self.history.append({"Student": user_message, "LLM Professor": response})
        return response
    
chatbot = ChatBot(retriever=chroma_retriever, llm=llm, prompt_template=chat_prompt_template)
print("LLM 수업 시작", end="\n\n")
while True:
    user_input = input("Student: ")
    if user_input.lower() in ["종료"]:
        print("LLM Professor: 오늘 수업은 여기서 마치겠습니다.", end="\n\n")
        print("LLM 수업 끝", end="\n\n")
        break
    response = chatbot.chat(user_input)
    print(f"Student: {user_input}", end="\n\n")
    print(f"LLM Professor: {response.content}", end="\n\n")

LLM 수업 시작

Student: 안녕하세요 교수님! 성함이 어떻게 되시나요?

LLM Professor: 안녕하세요! 저는 이화여자대학교에서 사이버보안학과 교수로 재직 중인 배 호입니다. 반갑습니다! 궁금한 점이나 도움이 필요하신 부분이 있다면 언제든지 말씀해 주세요.

Student: 배호 교수님이라고 부르면 될까요?

LLM Professor: 네, 저를 배호 교수님이라고 부르시면 됩니다! 언제든지 궁금한 점이나 도움이 필요하신 부분이 있다면 편하게 말씀해 주세요. 함께 공부하고 논의하는 것을 항상 환영합니다. 반갑습니다!

Student: 배호 교수님! 오늘 수업 주제는 무엇인가요?

LLM Professor: 안녕하세요! 오늘 수업의 주제는 Retrieval-Augmented Generation(RAG)입니다. RAG는 최신 정보를 외부 데이터베이스에서 검색하여 대형 언어 모델(LLM)의 응답을 생성하는 기술로, LLM의 환각 현상을 최소화하고 정보의 정확성을 높이는 데 중요한 역할을 합니다. 수업에서는 RAG의 기본 개념, 구조, 그리고 최신 연구 동향에 대해 다룰 예정입니다. 궁금한 점이 있다면 언제든지 질문해 주세요!

Student: rag가 어떻게 환각 현상을 최소화하지요?

LLM Professor: RAG는 외부 데이터베이스에서 최신 정보를 검색하여 LLM의 응답을 생성하는 방식으로 작동합니다. 이를 통해 LLM이 학습한 정보에만 의존하지 않고, 외부에서 확인된 사실적인 데이터에 기반하여 답변할 수 있습니다. 결과적으로, 잘못된 정보나 환각 현상이 발생할 가능성을 줄일 수 있습니다. 또한, RAG는 검색된 정보의 정확성과 관련성을 평가하여 더욱 신뢰할 수 있는 응답을 생성하는 데 도움을 줍니다. 잘 이해되었나요?

Student: rag와 데이터베이스의 관계에 대하여 설명해주세요

LLM Professor: RAG는 외부 데이터베이스에서 정보를 검색하여 LLM의 응답을 생성하는 기술입니다. 이 과정에서 데이터베이스는 RAG의 핵심 역할을

## 3 [My Code] Conclusion

1) 간단한 프롬프트와 클래스 선언 만으로 기본적인 챗봇이 구현되는 것을 확인하였습니다.

2) 적절한 질의응답을 통해 rag에 관하여 context 바깥의 질문을 던지면 답변을 안하는 것을 확인하였습니다. 이는 언어 모델의 Hallucinations를 방지하는 효과를 검증한 것입니다.

3) 일상적인 대화에 대해서는 gpt가 스스로 답변을 하는 것 또한 확인하였습니다. 이는 대화의 주제를 기반으로 모델 스스로 context 내의 주제인지 아닌지를 판단하고 있음을 확인한 것입니다.

4) 더 해볼 과제로는, 문장 생성과 임베딩 모델을 로컬 모델로 바꾸는 것과, 챗봇 구현 과정에서 세션을 구성하는 로직을 정교화하는 것입니다.